# Исследование опроса клиентов телекомунникацонной компании

**Описание проекта**

Заказчик данного исследования — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. Перед компанией стоит задача определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России.

Компания провела опрос и попросила вас подготовить дашборд с его итогами. Большую базу данных для такой задачи разворачивать не стали и выгрузили данные в SQLite.

Чтобы оценить результаты опроса, оценки обычно делят на три группы:

- **9-10 баллов** — «cторонники» (англ. promoters);
- **7-8 баллов** — «нейтралы» (англ. passives);
- **0-6 баллов** — «критики» (англ. detractors).

Итоговое значение NPS рассчитывается по формуле: % «сторонников» - % «критиков».

**Цель проекта**

Изучить предоставленные данные и ответить на следующие вопросы. Как распределены участники опроса по возрасту, полу и возрасту? Каких пользователей больше: новых или старых? Пользователи из каких городов активнее участвовали в опросе?

- Какие группы пользователей наиболее лояльны к сервису? Какие менее?

- Какой общий NPS среди всех опрошенных?

- Как можно описать клиентов, которые относятся к группе cторонников (англ. promoters)?

### Загрузка данных

In [3]:
import os
import pandas as pd
from sqlalchemy import create_engine

# путь к БД на вашем компьютере (например, в той же папке, что и тетрадь)
path_to_db_local ='F:/Обучение/Обучение програмированию/telecomm_csi.db'
# путь к БД на платформе
path_to_db_platform = '/datasets/telecomm_csi.db'
# итоговый путь к БД
path_to_db = None

# если путь на вашем компьютере ведёт к БД, то он становится итоговым
if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
# иначе: если путь на платформе ведёт к БД, то он становится итоговым
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
# иначе выводится сообщение о том, что файл не найден
else:
    raise Exception('Файл с базой данных SQLite не найден!')

# если итоговый путь не пустой
if path_to_db:
    # то создаём подключение к базе
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)
    
    # пример запроса
    query = """
SELECT u.user_id,
       u.lt_day,
       CASE 
           WHEN u.lt_day <= 365 THEN 'Новый'
           ELSE 'Старый'
           END AS is_new,
       u.age,
       CASE 
           WHEN u.gender_segment == 1 THEN 'Женщина'
           WHEN u.gender_segment == 0 THEN 'Мужчина'
           ELSE 'z'
           END AS gender_segment,
       u.os_name,
       u.cpe_type_name,
       location.country,
       location.city,
       SUBSTR(age_segment.title, 4) AS age_segment,
       SUBSTR(traffic_segment.title, 4) AS traffic_segment,
       SUBSTR(lifetime_segment.title, 4) AS lifetime_segment,
       u.nps_score,
       CASE 
           WHEN u.nps_score >= 9 THEN 'Cторонник'
           WHEN u.nps_score >= 7 THEN 'Нейтрал'
           ELSE 'Критик'
           END AS nps_group
FROM user AS u
JOIN location ON u.location_id = location.location_id
JOIN age_segment ON u.age_gr_id = age_segment.age_gr_id
JOIN traffic_segment ON u.tr_gr_id = traffic_segment.tr_gr_id
JOIN lifetime_segment ON u.lt_gr_id = lifetime_segment.lt_gr_id;
"""
    
    # создаём датафрейм по данным запроса
    df = pd.read_sql(query, engine) 

In [4]:
df.sample(10)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
377010,TI6O84,886,Старый,43.0,Женщина,IOS,SMARTPHONE,Россия,Екатеринбург,35-44,1-5,25-36,10,Cторонник
191345,JVYPF5,1078,Старый,24.0,Мужчина,PROPRIETARY,PHONE,Россия,Рязань,16-24,10-15,25-36,9,Cторонник
388233,U397G0,674,Старый,40.0,Женщина,ANDROID,SMARTPHONE,Россия,Ижевск,35-44,10-15,13-24,10,Cторонник
234679,M4TS6D,3575,Старый,59.0,Женщина,ANDROID,SMARTPHONE,Россия,Новосибирск,55-64,5-10,36+,10,Cторонник
357292,SHDQFT,446,Старый,34.0,Женщина,ANDROID,SMARTPHONE,Россия,Курск,25-34,20-25,13-24,5,Критик
106704,FHSSUB,191,Новый,38.0,Женщина,ANDROID,SMARTPHONE,Россия,Тула,35-44,10-15,7-12,10,Cторонник
486950,Z6SR7M,2866,Старый,42.0,Женщина,ANDROID,SMARTPHONE,Россия,Иркутск,35-44,0.1-1,36+,8,Нейтрал
287967,OW9OHH,795,Старый,35.0,Мужчина,ANDROID,SMARTPHONE,Россия,Москва,35-44,5-10,25-36,5,Критик
48776,CIOX5Z,178,Новый,30.0,Мужчина,IOS,SMARTPHONE,Россия,Новосибирск,25-34,30-35,4-6,5,Критик
267444,NU34D4,525,Старый,63.0,Мужчина,ANDROID,SMARTPHONE,Россия,Красноярск,55-64,55-60,13-24,10,Cторонник


### Описание данных

- **user_id** — идентификатор клиента 
- **lt_day** — количество дней «жизни» клиента 
- **is_new** — поле хранит информацию о том, является ли клиент новым 
- **age** — возраст gender_segment — пол os_name — тип операционной системы 
- **cpe_type_name** — тип устройства 
- **country** — страна проживания 
- **city** — город проживания 
- **age_segment** — возрастной сегмент 
- **traffic_segment** — сегмент по объёму потребляемого трафика 
- **lifetime_segment** — сегмент по количеству дней «жизни» nps_score — оценка клиента в NPS-опросе 
- **nps_group** — поле хранит информацию о том, к какой группе относится оценка клиента в опросе

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502493 entries, 0 to 502492
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           502493 non-null  object 
 1   lt_day            502493 non-null  int64  
 2   is_new            502493 non-null  object 
 3   age               501939 non-null  float64
 4   gender_segment    502493 non-null  object 
 5   os_name           502493 non-null  object 
 6   cpe_type_name     502493 non-null  object 
 7   country           502493 non-null  object 
 8   city              502493 non-null  object 
 9   age_segment       502493 non-null  object 
 10  traffic_segment   502493 non-null  object 
 11  lifetime_segment  502493 non-null  object 
 12  nps_score         502493 non-null  int64  
 13  nps_group         502493 non-null  object 
dtypes: float64(1), int64(2), object(11)
memory usage: 53.7+ MB


### Сохранение данных 

In [7]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

### Дашборд исследования NPS

[Ссылка на исследование уровня потребительской лояльности (уровень NPS) ver. 1.1.](https://public.tableau.com/app/profile/andrey.manannikov/viz/NPSManannikovA_A_/Story1?publish=yes)

### Исседование NPS в PDF формате 

[Ссылка на исселедования уровня потребительской лояльности (уровень NPS) ver. 1.1.](https://disk.yandex.ru/d/sBA5ThMDUa8LQg)